## Importing packages

In [1]:
import geemap
import ee
from geemap import chart
import matplotlib.pyplot as plt

In [4]:
ee.Initialize()

EEException: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, and then retry.

##  Change Detection

In [3]:
admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
thane = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Thane'))
Map = geemap.Map()


s2 = ee.ImageCollection("COPERNICUS/S2")

rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

# Write a function for Cloud masking
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.date('2019-01-01', '2019-12-31')) \
  .filter(ee.Filter.bounds(thane)) \
  .map(maskS2clouds)

timelapseimage = s2 \
  .filter(ee.Filter.bounds(thane)) \
  .map(maskS2clouds)

filtered2 = s2 \
  .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
  .filter(ee.Filter.bounds(thane)) \
  .map(maskS2clouds)

image2019 = filtered.median().clip(thane)
image2020 = filtered2.median().clip(thane)

# Display the input composite.

Map.addLayer(image2019, rgbVis, '2019')
Map.addLayer(image2020, rgbVis, '2020')

ndvi2019 = image2019.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
ndvi2020 = image2020.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
ndviVis = {'min':0, 'max':1, 'palette': ['white', 'green']}

Map.addLayer(ndvi2019, ndviVis, 'ndvi')
Map.addLayer(ndvi2020, ndviVis, 'ndvi')


# 3. Classify NDVI into 5 classes
ndvi2019classify = ee.Image(1) \
          .where(ndvi2019.gt(0.0).And(ndvi2019.lte(0.2)), 2) \
          .where(ndvi2019.gt(0.2).And(ndvi2019.lte(0.4)), 3) \
          .where(ndvi2019.gt(0.4).And(ndvi2019.lte(0.6)), 4) \
          .where(ndvi2019.gt(0.6), 5).clip(thane)

ndvi2020classify = ee.Image(1) \
          .where(ndvi2020.gt(0.0).And(ndvi2020.lte(0.2)), 2) \
          .where(ndvi2020.gt(0.2).And(ndvi2020.lte(0.4)), 3) \
          .where(ndvi2020.gt(0.4).And(ndvi2020.lte(0.6)), 4) \
          .where(ndvi2020.gt(0.6), 5).clip(thane)

# Add map layers
Map.addLayer(ndvi2019classify, {'min': 1, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Classified NDVI 2019',True)
Map.addLayer(ndvi2020classify, {'min': 1, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Classified NDVI 2020',True)


#spectral changes
def magnitude(image):
  return image.pow(2).reduce(ee.Reducer.sum()).sqrt()

distance = ndvi2020.subtract(ndvi2019)


Map.addLayer(distance, {'min': 0, 'max': 0.1, 'palette': ['white', 'green']}, 'spectral distance')
Map.centerObject(thane, 9)
Map


EEException: Earth Engine client library not initialized. Run `ee.Initialize()`

In [8]:
threshold=0.3

changes = distance.lte(threshold).Or(distance.gte(threshold)).selfMask()
# Calculating area above the threshold value
pixel_area=changes.multiply(ee.Image.pixelArea())
area=pixel_area.reduceRegion(reducer=ee.Reducer.sum(), geometry=thane, scale=30 ,maxPixels=10e9)

print(area.get('NDVI'))

ee.Dictionary({
  "functionInvocationValue": {
    "functionName": "Image.reduceRegion",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "Collection.geometry",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.loadTable",
                      "arguments": {
                        "tableId": {
                          "constantValue": "FAO/GAUL_SIMPLIFIED_500m/2015/level2"
                        }
                      }
                    }
                  },
                  "filter": {
                    "functionInvocationValue": {
                      "functionName": "Filter.equals",
                      "arguments": {
                        "leftField": {
  

In [1]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling: voila
- Writing config: /Users/harsh/miniforge3/envs/mlp/etc/jupyter
    - Validating...
      voila 0.3.5 OK
